In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Etude du lien entre les différentes bases de données : une approche économétrique

On cherche à étudier le lien entre les différentes séries dont on dispose et le nombre officiel de touristes. Pour cela, on commence par s'assurer que les données à comparer soient comparables.

## Import des bases de données

### Données trafic aérien France

In [2]:
df_air_trafic = pd.read_csv("df_air_trafic_corrigee.csv", index_col=0)
df_air_trafic

,country_origin,citydest,sheduled_time_start,seats,number_planes,Year,Month,Day,Year-Month
64488,Hungary,Paris,2017-05-16,462.0,3,2017,5,16,2017-05
64345,Germany,Basel,2017-05-16,592.0,5,2017,5,16,2017-05
64346,France,Toulon,2017-05-16,334.0,3,2017,5,16,2017-05
64347,France,Bordeaux,2017-05-16,1736.0,15,2017,5,16,2017-05
64348,Portugal,Toulouse,2017-05-16,437.0,3,2017,5,16,2017-05
...,...,...,...,...,...,...,...,...,...
265668,Germany,Paris,2021-12-07,1314.0,9,2021,12,7,2021-12
265666,Portugal,Paris,2021-12-07,183.0,2,2021,12,7,2021-12
265684,France,Nice,2021-12-07,1387.0,9,2021,12,7,2021-12
265672,Tunisia,Paris,2021-12-07,298.0,2,2021,12,7,2021-12


Comme les données officielles recensent le nombre de touristes arrivés en France par mois, le but est d'obtenir un nombre de vols et de sièges par mois.

In [3]:
df_air_trafic_month = pd.read_csv("df_air_trafic_corrigee_month.csv", index_col=0)
df_air_trafic_month['sheduled_time_start'] = pd.to_datetime(df_air_trafic_month['sheduled_time_start'])
df_air_trafic_month

,sheduled_time_start,citydest,seats,number_planes
0,2017-05-01,Agen,372.0,7
1,2017-05-01,Ajaccio,21455.0,147
2,2017-05-01,Basel,102306.0,682
3,2017-05-01,Bastia,16665.0,119
4,2017-05-01,Bergerac,6072.0,46
...,...,...,...,...
2100,2021-12-01,Rodez,350.0,7
2101,2021-12-01,Strasbourg,3730.0,30
2102,2021-12-01,Tarbes,189.0,1
2103,2021-12-01,Toulouse,22385.0,149


In [36]:
#En groupant sur la France entière, sans dissocier par départements :
df_air_trafic_month2 = df_air_trafic_month.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()
df_air_trafic_month2.reset_index(inplace=True)
df_air_trafic_month2.head(5)

/var/folders/hl/40sn85xd679cr7nqv4v_jsrh0000gn/T/ipykernel_79446/2500076060.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_air_trafic_month2 = df_air_trafic_month.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()


,sheduled_time_start,seats,number_planes
0,2017-05-01,2277076.0,13637
1,2017-06-01,4244152.0,25250
2,2017-07-01,4883670.0,28912
3,2017-08-01,4964357.0,28874
4,2017-09-01,4814087.0,28821


### Données officielles : arrivées dans l'hôtellerie

In [5]:
df_France_Official_Data = pd.read_csv('France_Official_Data.csv')

df_France_Official_Data['date']=pd.to_datetime(df_France_Official_Data['date'])
df_France_Official_Data['value_in_thousands']=pd.to_numeric(df_France_Official_Data['value_in_thousands'], errors='coerce')

In [6]:
#Attention : bcp de valeurs manquantes en 2020
df_France_Official_Data[df_France_Official_Data["date"]=="2020-01-01"]

,date,variable,department,individus,value_in_thousands
108,2020-01-01,Arrivées dans l'hôtellerie,Ain,Non-résidents,NaN
238,2020-01-01,Arrivées dans l'hôtellerie,Ain,Résidents,NaN
368,2020-01-01,Arrivées dans l'hôtellerie,Aisne,Non-résidents,NaN
498,2020-01-01,Arrivées dans l'hôtellerie,Aisne,Résidents,NaN
628,2020-01-01,Arrivées dans l'hôtellerie,Allier,Non-résidents,NaN
...,...,...,...,...,...
24418,2020-01-01,Arrivées dans l'hôtellerie,Vosges,Résidents,NaN
24548,2020-01-01,Arrivées dans l'hôtellerie,Yonne,Non-résidents,NaN
24678,2020-01-01,Arrivées dans l'hôtellerie,Yonne,Résidents,NaN
24808,2020-01-01,Arrivées dans l'hôtellerie,Yvelines,Non-résidents,NaN


In [7]:
df_France_Official_Data2 = pd.read_csv("df_France_Official_Data_month.csv", index_col=0)
df_France_Official_Data2

,date,department,value_in_thousands
0,2011-01-01,Ain,47.32
1,2011-01-01,Aisne,21.78
2,2011-01-01,Allier,23.62
3,2011-01-01,Alpes-Maritimes,170.80
4,2011-01-01,Alpes-de-Haute-Provence,15.27
...,...,...,...
12475,2021-10-01,Vendée,82.39
12476,2021-10-01,Vienne,96.87
12477,2021-10-01,Vosges,39.99
12478,2021-10-01,Yonne,41.35


In [8]:
#En groupant sur la France entière, sans dissocier par départements :
df_France_Official_Data_month = df_France_Official_Data.groupby('date', as_index=False).sum() 
df_France_Official_Data_month

,date,value_in_thousands
0,2011-01-01,6380.57
1,2011-02-01,6707.55
2,2011-03-01,8112.46
3,2011-04-01,9329.19
4,2011-05-01,9917.42
...,...,...
125,2021-06-01,6913.27
126,2021-07-01,10498.30
127,2021-08-01,11095.90
128,2021-09-01,9442.13


### Commentaires sur les hôtels

In [45]:
df_reviews_us_france_210712 = pd.read_csv('reviews_us_france_210712.csv')

df_reviews_france=df_reviews_us_france_210712[df_reviews_us_france_210712['country']=="France"]

df_reviews_france['stay_date']=pd.to_datetime(df_reviews_france['stay_date'])
df_reviews_france['publication_date']=pd.to_datetime(df_reviews_france['publication_date'])

df_reviews_france

/var/folders/hl/40sn85xd679cr7nqv4v_jsrh0000gn/T/ipykernel_79446/3293369237.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_france['stay_date']=pd.to_datetime(df_reviews_france['stay_date'])
/var/folders/hl/40sn85xd679cr7nqv4v_jsrh0000gn/T/ipykernel_79446/3293369237.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_france['publication_date']=pd.to_datetime(df_reviews_france['publication_date'])


,country,city,reviewerLocation,publication_date,stay_date,number_of_reviewers
77,France,Paris,France,2021-11-14,2021-11-01,87
163,France,Paris,France,2021-11-22,2021-11-01,69
179,France,Paris,France,2021-11-29,2021-11-01,67
189,France,Paris,France,2021-11-28,2021-11-01,65
232,France,Paris,France,2021-11-15,2021-11-01,59
...,...,...,...,...,...,...
119223,France,Paris,Turkey,2018-08-07,2018-07-01,1
119224,France,Nice,Poland,2021-07-19,2021-06-01,1
119225,France,Paris,Turkey,2018-08-05,2018-08-01,1
119226,France,Strasbourg,Botswana,2021-07-14,2021-07-01,1


In [10]:
df_reviews_france2 = df_reviews_france.groupby("stay_date", as_index=False).sum()
df_reviews_france2

,stay_date,number_of_reviewers
0,2018-07-01,936
1,2018-08-01,4342
2,2018-09-01,3614
3,2018-10-01,3477
4,2018-11-01,3755
5,2018-12-01,3842
6,2019-01-01,3148
7,2019-02-01,2856
8,2019-03-01,3254
9,2019-04-01,3936


### Données google trends

In [11]:
df_gt=pd.read_csv("google_trends_fr_1.csv")

df_gt['date']=pd.to_datetime(df_gt['date'])

df_gt2 = df_gt.reset_index().pivot_table(values = "value", index = "date", columns = "variable")
df_gt2=df_gt2.reset_index()

Le même problème que précédemment se pose: les données sont ici journalières. Comme les données officielles sont mensuelles, il faut faire une moyenne mensuelle ici - les données représentent cette fois ci non un total comme le nombre de vols ou de sièges mais sont un indice. Faire une somme n'aurait alors pas de sens.

In [12]:
df_gg_trends_month = pd.read_csv("df_gg_trends_month.csv", index_col=0)
df_gg_trends_month['Year-Month']=pd.to_datetime(df_gg_trends_month['Year-Month'])

df_gg_trends_month

,aeroport nice cote d azur theme_cat_voyages,aeroport paris_cat_voyages,cote d azur theme_cat_voyages,disneyland paris hotel_cat_voyages,disneyland paris theme_cat_voyages,disneyland paris_cat_voyages,french riviera_cat_voyages,louvre_cat_voyages,lyon gare_cat_voyages,lyon hotel_cat_voyages,...,nice hotel_cat_voyages,nice_cat_voyages,paris gare_cat_voyages,paris hotel_cat_voyages,paris restaurant_cat_voyages,paris_cat_voyages,promenade des anglais_cat_voyages,Year-Month,Year,Month
0,0.380821,0.397343,0.218335,0.658764,0.675476,0.703867,0.113660,0.453864,0.402707,0.736345,...,0.375507,0.311773,0.450731,0.727737,0.746280,0.688695,0.042035,2017-01-01,2017.0,1.0
1,0.397884,0.668512,0.286189,0.444575,0.637644,0.631116,0.176885,0.163524,0.521349,0.688254,...,0.649218,0.435043,0.860878,0.723169,0.555221,0.647908,0.048989,2017-02-01,2017.0,2.0
2,0.581424,0.175265,0.382997,0.592372,0.615289,0.621114,0.470835,0.712439,0.482256,0.644683,...,0.517624,0.521424,0.437975,0.706606,0.513693,0.615938,0.055310,2017-03-01,2017.0,3.0
3,0.547968,0.654394,0.463015,0.357520,0.748321,0.593453,0.830587,0.586943,0.483234,0.617882,...,0.481717,0.558308,0.576582,0.674831,0.596839,0.746915,0.079215,2017-04-01,2017.0,4.0
4,0.556246,0.682465,0.466528,0.354153,0.595350,0.660548,0.481935,0.419978,0.512461,0.528303,...,0.499634,0.510721,0.618393,0.582283,0.522685,0.752421,0.116150,2017-05-01,2017.0,5.0
5,0.571655,0.676586,0.872862,0.296662,0.575806,0.532279,0.704010,0.406900,0.620544,0.574003,...,0.652771,0.679630,0.556987,0.546576,0.724417,0.795421,0.357723,2017-06-01,2017.0,6.0
6,0.817660,1.000000,1.000000,0.567889,0.771065,0.658632,0.478763,0.495630,0.656326,0.753046,...,1.000000,0.775119,0.680724,0.686342,0.585619,0.791478,0.754093,2017-07-01,2017.0,7.0
7,0.728079,0.777475,0.667771,0.631959,0.826268,0.789657,0.649904,0.628138,0.550618,0.657217,...,0.503686,0.711338,0.602542,0.667611,0.815195,0.711409,0.760398,2017-08-01,2017.0,8.0
8,0.782077,0.761938,0.280600,0.543303,0.716172,0.685639,0.323049,0.571114,0.553934,0.724764,...,0.546383,0.468453,0.605469,0.766857,0.817579,0.738793,0.157229,2017-09-01,2017.0,9.0
9,0.503685,0.659248,0.280885,0.569772,0.807996,0.768136,0.146393,0.634673,0.587284,0.854749,...,0.469290,0.431135,0.666219,0.854239,0.703367,0.694941,0.000000,2017-10-01,2017.0,10.0


## Fusion des df

In [13]:
# Fusion du nb d'avions et du nb de sièges et le nb de touristes offciels
df_fusion_aerien = df_France_Official_Data_month.merge(df_air_trafic_month2, how='inner', left_on="date", right_on="sheduled_time_start")
df_fusion_aerien.head(5)

,date,value_in_thousands,sheduled_time_start,seats,number_planes
0,2017-05-01,10382.97,2017-05-01,2277076.0,13637
1,2017-06-01,11560.11,2017-06-01,4244152.0,25250
2,2017-07-01,12972.80,2017-07-01,4883670.0,28912
3,2017-08-01,12835.78,2017-08-01,4964357.0,28874
4,2017-09-01,11343.99,2017-09-01,4814087.0,28821


In [14]:
# Fusion des commentaires laissés par les touristes et le nb de touristes offciels
df_fusion_commentaires = df_France_Official_Data_month.merge(df_reviews_france2, how='inner', left_on="date", right_on="stay_date")
df_fusion_commentaires.head(5)

,date,value_in_thousands,stay_date,number_of_reviewers
0,2018-07-01,12938.61,2018-07-01,936
1,2018-08-01,13230.29,2018-08-01,4342
2,2018-09-01,11500.99,2018-09-01,3614
3,2018-10-01,10298.17,2018-10-01,3477
4,2018-11-01,8161.55,2018-11-01,3755


In [15]:
# Fusion des gt et le nb de touristes offciels
df_fusion_gt=df_France_Official_Data_month.merge(df_gg_trends_month, how='inner', left_on="date", right_on="Year-Month")
df_fusion_gt

,date,value_in_thousands,aeroport nice cote d azur theme_cat_voyages,aeroport paris_cat_voyages,cote d azur theme_cat_voyages,disneyland paris hotel_cat_voyages,disneyland paris theme_cat_voyages,disneyland paris_cat_voyages,french riviera_cat_voyages,louvre_cat_voyages,...,nice hotel_cat_voyages,nice_cat_voyages,paris gare_cat_voyages,paris hotel_cat_voyages,paris restaurant_cat_voyages,paris_cat_voyages,promenade des anglais_cat_voyages,Year-Month,Year,Month
0,2017-01-01,6887.63,0.380821,0.397343,0.218335,0.658764,0.675476,0.703867,0.113660,0.453864,...,0.375507,0.311773,0.450731,0.727737,0.746280,0.688695,0.042035,2017-01-01,2017.0,1.0
1,2017-02-01,7304.45,0.397884,0.668512,0.286189,0.444575,0.637644,0.631116,0.176885,0.163524,...,0.649218,0.435043,0.860878,0.723169,0.555221,0.647908,0.048989,2017-02-01,2017.0,2.0
2,2017-03-01,8564.83,0.581424,0.175265,0.382997,0.592372,0.615289,0.621114,0.470835,0.712439,...,0.517624,0.521424,0.437975,0.706606,0.513693,0.615938,0.055310,2017-03-01,2017.0,3.0
3,2017-04-01,10106.67,0.547968,0.654394,0.463015,0.357520,0.748321,0.593453,0.830587,0.586943,...,0.481717,0.558308,0.576582,0.674831,0.596839,0.746915,0.079215,2017-04-01,2017.0,4.0
4,2017-05-01,10382.97,0.556246,0.682465,0.466528,0.354153,0.595350,0.660548,0.481935,0.419978,...,0.499634,0.510721,0.618393,0.582283,0.522685,0.752421,0.116150,2017-05-01,2017.0,5.0
5,2017-06-01,11560.11,0.571655,0.676586,0.872862,0.296662,0.575806,0.532279,0.704010,0.406900,...,0.652771,0.679630,0.556987,0.546576,0.724417,0.795421,0.357723,2017-06-01,2017.0,6.0
6,2017-07-01,12972.80,0.817660,1.000000,1.000000,0.567889,0.771065,0.658632,0.478763,0.495630,...,1.000000,0.775119,0.680724,0.686342,0.585619,0.791478,0.754093,2017-07-01,2017.0,7.0
7,2017-08-01,12835.78,0.728079,0.777475,0.667771,0.631959,0.826268,0.789657,0.649904,0.628138,...,0.503686,0.711338,0.602542,0.667611,0.815195,0.711409,0.760398,2017-08-01,2017.0,8.0
8,2017-09-01,11343.99,0.782077,0.761938,0.280600,0.543303,0.716172,0.685639,0.323049,0.571114,...,0.546383,0.468453,0.605469,0.766857,0.817579,0.738793,0.157229,2017-09-01,2017.0,9.0
9,2017-10-01,10065.70,0.503685,0.659248,0.280885,0.569772,0.807996,0.768136,0.146393,0.634673,...,0.469290,0.431135,0.666219,0.854239,0.703367,0.694941,0.000000,2017-10-01,2017.0,10.0


# Test de régression

In [16]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

## En prenant chaque df de manière séparée

In [17]:
# Nb de vols
X = sm.add_constant(df_fusion_aerien["number_planes"])
results = sm.OLS(df_fusion_aerien['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.648
Method:                 Least Squares   F-statistic:                     98.53
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           1.34e-13
Time:                        11:11:39   Log-Likelihood:                -502.02
No. Observations:                  54   AIC:                             1008.
Df Residuals:                      52   BIC:                             1012.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           263.6266    823.572      0.320

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [18]:
# Nb de sièges
X = sm.add_constant(df_fusion_aerien["seats"])
results = sm.OLS(df_fusion_aerien['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     92.50
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           3.93e-13
Time:                        11:11:39   Log-Likelihood:                -503.13
No. Observations:                  54   AIC:                             1010.
Df Residuals:                      52   BIC:                             1014.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        160.6606    857.795      0.187      0.8

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [19]:
# Nb de commentaires
X = sm.add_constant(df_fusion_commentaires["number_of_reviewers"])
results = sm.OLS(df_fusion_commentaires['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     25.39
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           1.18e-05
Time:                        11:11:39   Log-Likelihood:                -385.58
No. Observations:                  40   AIC:                             775.2
Df Residuals:                      38   BIC:                             778.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1479.9236   1

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [20]:
# Google trends
X = sm.add_constant(df_fusion_gt.drop(["date", "value_in_thousands", "Year-Month", "Year", "Month"], axis = 1))
results = sm.OLS(df_fusion_gt['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     8.221
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           6.25e-08
Time:                        11:11:39   Log-Likelihood:                -508.25
No. Observations:                  58   AIC:                             1070.
Df Residuals:                      31   BIC:                             1126.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


On met en log car les données des google trends c'est un indice donc ce sera plus représentatif d'interpréter en log.

In [21]:
columns = list(df_fusion_gt.columns.values)
columns.remove("date")
columns.remove("Year-Month")

df_fusion_gt_log = np.log(df_fusion_gt[columns]) #transforme toutes les variables en log
df_fusion_gt_log.replace([np.inf, -np.inf], np.nan, inplace=True) 
df_fusion_gt_log.dropna(inplace=True) 


X = sm.add_constant(df_fusion_gt_log.drop(["value_in_thousands", "Year", "Month"], axis = 1))
results = sm.OLS(df_fusion_gt_log['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     6.209
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           0.000297
Time:                        11:11:39   Log-Likelihood:                 53.232
No. Observations:                  42   AIC:                            -52.46
Df Residuals:                      15   BIC:                            -5.548
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# Régression sur l'ensemble des variables

In [22]:
df_fusion1=df_fusion_aerien.merge(df_fusion_commentaires, how='inner', on="date")
df_fusion1=df_fusion1[["date", "seats", "number_planes", "number_of_reviewers"]]

In [23]:
df_fusion2=df_fusion1.merge(df_fusion_gt, how='inner', on="date")

In [24]:
df_fusion2

,date,seats,number_planes,number_of_reviewers,value_in_thousands,aeroport nice cote d azur theme_cat_voyages,aeroport paris_cat_voyages,cote d azur theme_cat_voyages,disneyland paris hotel_cat_voyages,disneyland paris theme_cat_voyages,...,nice hotel_cat_voyages,nice_cat_voyages,paris gare_cat_voyages,paris hotel_cat_voyages,paris restaurant_cat_voyages,paris_cat_voyages,promenade des anglais_cat_voyages,Year-Month,Year,Month
0,2018-07-01,4883670.0,28912,936,12938.61,0.811025,0.762429,0.973548,0.706928,0.685507,...,0.909569,0.810995,0.637424,0.798625,0.643405,0.851685,1.000000,2018-07-01,2018.0,7.0
1,2018-08-01,4964357.0,28874,4342,13230.29,0.949429,0.709878,0.898167,0.612211,0.886331,...,0.597335,0.809900,0.686517,0.749312,0.716532,0.776532,0.328319,2018-08-01,2018.0,8.0
2,2018-09-01,4814087.0,28821,3614,11500.99,0.582595,0.575087,0.346928,0.834393,0.751606,...,0.497325,0.561669,0.834177,1.000000,0.615217,0.805614,0.017146,2018-09-01,2018.0,9.0
3,2018-10-01,4780669.0,28277,3477,10298.17,0.628858,0.343900,0.086162,0.746295,0.748025,...,0.480968,0.542930,0.838292,0.755408,0.893131,0.758342,0.070465,2018-10-01,2018.0,10.0
4,2018-11-01,3670860.0,21379,3755,8161.55,0.478853,0.484567,0.060377,0.454566,0.690657,...,0.352056,0.310603,0.677242,0.685019,0.832980,0.733972,0.219469,2018-11-01,2018.0,11.0
5,2018-12-01,3709832.0,20943,3842,7995.92,0.439817,0.604286,0.124054,0.318111,0.528018,...,0.241653,0.373445,0.524666,0.469029,0.647797,0.630759,0.140542,2018-12-01,2018.0,12.0
6,2019-01-01,3553036.0,20469,3148,7020.50,0.463821,0.743632,0.329892,0.682976,0.749738,...,0.532201,0.483514,0.555904,0.826824,0.896668,0.816628,0.038717,2019-01-01,2019.0,1.0
7,2019-02-01,3562193.0,20856,2856,7582.23,0.450518,0.541473,0.279813,0.527054,0.635047,...,0.507278,0.539366,0.678186,0.624872,0.822045,0.771003,0.024494,2019-02-01,2019.0,2.0
8,2019-03-01,3984509.0,23365,3254,8979.44,0.265794,0.628530,0.388322,0.704522,0.346762,...,0.417001,0.479835,0.811902,0.717844,0.833625,0.777738,0.115763,2019-03-01,2019.0,3.0
9,2019-04-01,4609542.0,27047,3936,10352.06,0.844073,0.944821,0.687616,0.499837,0.841196,...,0.549860,0.645223,0.884245,0.739918,0.910803,0.463509,0.353665,2019-04-01,2019.0,4.0


Ici, on fait la régression du nb total de touristes sur toutes nos variables: pas forcément pertinent. trop de variables?

In [25]:
# Régression sur la base finale
X = sm.add_constant(df_fusion2.drop(["date", "value_in_thousands", "Year", "Month", "Year-Month"], axis = 1))
results = sm.OLS(df_fusion2['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     16.24
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           3.07e-05
Time:                        11:11:39   Log-Likelihood:                -318.35
No. Observations:                  40   AIC:                             696.7
Df Residuals:                      10   BIC:                             747.4
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Ici on fait la régression du nb total de touristes sur toutes nos variables sans les GG trends pour voir les coeff significatifs :

In [26]:
df_fusion_sansGGtrend = df_fusion_aerien.merge(df_fusion_commentaires, how='inner', on="date")
df_fusion_sansGGtrend = df_fusion_sansGGtrend[["date", "value_in_thousands_x", "seats", "number_planes", "number_of_reviewers"]]
df_fusion_sansGGtrend.rename(columns={"value_in_thousands_x": "value_in_thousands"}, inplace=True)
df_fusion_sansGGtrend.head(5)

,date,value_in_thousands,seats,number_planes,number_of_reviewers
0,2018-07-01,12938.61,4883670.0,28912,936
1,2018-08-01,13230.29,4964357.0,28874,4342
2,2018-09-01,11500.99,4814087.0,28821,3614
3,2018-10-01,10298.17,4780669.0,28277,3477
4,2018-11-01,8161.55,3670860.0,21379,3755


In [27]:
# Régression sur la base finale
X = sm.add_constant(df_fusion_sansGGtrend.drop(["date", "value_in_thousands"], axis = 1))
results = sm.OLS(df_fusion_sansGGtrend['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     46.64
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           1.74e-12
Time:                        11:11:39   Log-Likelihood:                -364.08
No. Observations:                  40   AIC:                             736.2
Df Residuals:                      36   BIC:                             742.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                -994.0986    

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


**ANALYSE PAR DEPARTEMENT**

On restreint les données officielles, du trafic aériens et les reviews à 1 département.
On fusionne les bases et on fait les régressions.

1) Alpes Maritimes (Nice)

In [112]:
df_Official_Data_AM = df_France_Official_Data[(df_France_Official_Data['department']=='Alpes-Maritimes')]
df_Official_Data_AM2 = df_Official_Data_AM.groupby('date').sum() 
#df_Official_Data_AM2.head(5)

In [113]:
df_air_trafic_month_NICE = df_air_trafic_month[(df_air_trafic_month['citydest']=='Nice')]
df_air_trafic_month_NICE2 = df_air_trafic_month_NICE.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()
df_air_trafic_month_NICE2.reset_index(inplace=True)
#df_air_trafic_month_NICE2.head(5)

/var/folders/hl/40sn85xd679cr7nqv4v_jsrh0000gn/T/ipykernel_79446/2833740425.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_air_trafic_month_NICE2 = df_air_trafic_month_NICE.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()


In [114]:
df_reviews_france_AM=df_reviews_france[df_reviews_france['city']=="Nice"]
df_reviews_france_AM2 = df_reviews_france_AM.groupby("stay_date", as_index=False).sum()
#df_reviews_france_AM2.head(5)

In [116]:
df_fusion_AM = df_Official_Data_AM2.merge(df_air_trafic_month_NICE2, how='inner', left_on="date", right_on="sheduled_time_start")
#df_fusion_AM.head(5)

In [115]:
df_fusion_commentaires_AM = df_Official_Data_AM2.merge(df_reviews_france_AM2, how='inner', left_on="date", right_on="stay_date")
#df_fusion_commentaires_AM.head(5)

In [71]:
X = sm.add_constant(df_fusion_AM.drop(["value_in_thousands", "sheduled_time_start"], axis = 1))
results = sm.OLS(df_fusion_AM['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     87.47
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           3.28e-17
Time:                        14:49:30   Log-Likelihood:                -313.97
No. Observations:                  54   AIC:                             633.9
Df Residuals:                      51   BIC:                             639.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -6.8403     25.439     -0.269

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [70]:
X = sm.add_constant(df_fusion_commentaires_AM["number_of_reviewers"])
results = sm.OLS(df_fusion_commentaires_AM['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.565
Method:                 Least Squares   F-statistic:                     32.23
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           8.84e-06
Time:                        14:48:31   Log-Likelihood:                -150.72
No. Observations:                  25   AIC:                             305.4
Df Residuals:                      23   BIC:                             307.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  29.5841    

2) Rhône (Lyon)

In [109]:
df_Official_Data_rhone = df_France_Official_Data[(df_France_Official_Data['department']=='Rhône')]
df_Official_Data_rhone2 = df_Official_Data_rhone.groupby('date').sum() 
#df_Official_Data_rhone2.head(6)

In [110]:
df_air_trafic_month_rhone = df_air_trafic_month[(df_air_trafic_month['citydest']=='Lyon')]
df_air_trafic_month_rhone2 = df_air_trafic_month_rhone.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()
df_air_trafic_month_rhone2.reset_index(inplace=True)
#df_air_trafic_month_rhone2.head(5)

/var/folders/hl/40sn85xd679cr7nqv4v_jsrh0000gn/T/ipykernel_79446/1511331543.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_air_trafic_month_Corse2 = df_air_trafic_month_Corse.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()


In [111]:
df_reviews_france_rhone=df_reviews_france[df_reviews_france['city']=="Lyon"]
df_reviews_france_rhone2 = df_reviews_france_rhone.groupby("stay_date", as_index=False).sum()
#df_reviews_france_rhone2.head(5)

In [106]:
df_fusion_rhone = df_Official_Data_rhone2.merge(df_air_trafic_month_rhone2, how='inner', left_on="date", right_on="sheduled_time_start")

df_fusion_commentaires_rhone = df_Official_Data_rhone2.merge(df_reviews_france_rhone2, how='inner', left_on="date", right_on="stay_date")

In [107]:
X = sm.add_constant(df_fusion_rhone.drop(["value_in_thousands", "sheduled_time_start"], axis = 1))
results = sm.OLS(df_fusion_rhone['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     37.45
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           9.84e-11
Time:                        18:11:37   Log-Likelihood:                -315.83
No. Observations:                  54   AIC:                             637.7
Df Residuals:                      51   BIC:                             643.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            58.7404     24.100      2.437

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [108]:
X = sm.add_constant(df_fusion_commentaires_rhone["number_of_reviewers"])
results = sm.OLS(df_fusion_commentaires_rhone['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     20.84
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           5.12e-05
Time:                        18:11:44   Log-Likelihood:                -246.40
No. Observations:                  40   AIC:                             496.8
Df Residuals:                      38   BIC:                             500.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  24.9857    

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


3) Gironde (Bordeaux)

In [117]:
df_Official_Data_gironde = df_France_Official_Data[(df_France_Official_Data['department']=='Gironde')]
df_Official_Data_gironde2 = df_Official_Data_gironde.groupby('date').sum() 
#df_Official_Data_gironde2.head(6)

In [118]:
df_air_trafic_month_gironde = df_air_trafic_month[(df_air_trafic_month['citydest']=='Bordeaux')]
df_air_trafic_month_gironde2 = df_air_trafic_month_gironde.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()
df_air_trafic_month_gironde2.reset_index(inplace=True)
#df_air_trafic_month_gironde2.head(5)

/var/folders/hl/40sn85xd679cr7nqv4v_jsrh0000gn/T/ipykernel_79446/1449040169.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_air_trafic_month_gironde2 = df_air_trafic_month_gironde.groupby(pd.Grouper(key='sheduled_time_start', freq="MS"))['seats', 'number_planes'].sum()


In [119]:
df_reviews_france_gironde=df_reviews_france[df_reviews_france['city']=="Bordeaux"]
df_reviews_france_gironde2 = df_reviews_france_gironde.groupby("stay_date", as_index=False).sum()
#df_reviews_france_gironde2.head(5)

In [120]:
df_fusion_gironde = df_Official_Data_gironde2.merge(df_air_trafic_month_gironde2, how='inner', left_on="date", right_on="sheduled_time_start")


In [121]:
X = sm.add_constant(df_fusion_gironde.drop(["value_in_thousands", "sheduled_time_start"], axis = 1))
results = sm.OLS(df_fusion_gironde['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.671
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     52.11
Date:                Wed, 16 Mar 2022   Prob (F-statistic):           4.72e-13
Time:                        18:22:22   Log-Likelihood:                -301.54
No. Observations:                  54   AIC:                             609.1
Df Residuals:                      51   BIC:                             615.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            25.8094     18.474      1.397

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
